The unoptimal solution to this problem is trying all possible values for k between two words to decide if their compatibility. This method is brute force. To determine the complexity, we consider the worst possible scenario. We note:

n - the number of words

m - the length of the words

In this context, we need the following operations:

*   Shifting the characters in the word with k positions => O(m)
*   Comparing two words => O(m)
*   Try all 26 possible rotations => O(1)
*   Comparing all words and all belong to a different equivalence class =>

$$O\left( \frac{n \cdot (n-1)}{2}\right) = O\left(n^2\right)$$

The total complexity that we get is:
$$O\left(2m \cdot n^2\right) = O\left(m \cdot n^2\right)$$

The solution implemented and tested below reduces this complexity by determining if two words are class equivalent in a linear manner. We can get from one $word_{i}$ to $word_{j}$ with a finite number of rotations, if the two have the exact same pattern of difference between the two consecutive letters. For example, the words 'HELLO' and 'IFMMP' are calss equivalent because both form the pattern (3, -7, 0, -3) (i.e. 'H' - 'E' = 'I' - 'F'= 3, 'E'-'L' = 'F' - M' = -7, 'L'-'L' = 'M - 'M' = 0 etc.)

For this, we need the following operations:

*   Calculating the pattern for each word =>
$$O\left((m-1) \cdot n\right) = O\left(m \cdot n\right)$$

*   Creating and updating the dictiobary => O(n)

The total complexity that we get is:
$$ O\left(m \cdot n + n \right) =O\left(m \cdot n\right)$$


## Utils

In [ ]:
#Here we determine the size of the english alphabet
print(ord('Z') - ord('A')+1)

26


In [73]:
#This method was used initially for the unoptimal solution
#Then it was used to generate tests
#this method takes as input the word and number of positions to be shifted
#
def rot(word: str ,k:int)->str:
  """
  This function applies a basic rotation (Caesar cipher) to the input string.

  Parameters:
  - word (str): The input word to be encrypted. Assumes all uppercase letters.
  - k (int): The number of positions each letter in the input word is shifted.

  Returns:
  - str: The encrypted word after applying the Caesar cipher rotation.

  Note: The function is designed to work with uppercase letters only and does not account for non-alphabetical
  characters.
  """
  #shifting an input with the size of the alphabet yields the input itself
  if k > 26:
    k=k%26

  #building the alphabet
  alpha = [chr(ord('A') + i) for i in range(ord('Z') - ord('A')+1)]
  caes = []
  for ch in word:

    # Finding the encrypted character and append it to the 'caes' list
    caes.append(alpha[(ord(ch)-ord('A')+k)%26]) # by applying % alphabet_size, we ensure that the pattern is not affected by the circularity of the letter sequence. Rotating 'Z' by 1 would result in 'A'

  #retruning the encrypted word
  return ''.join(caes)

In [75]:
print(rot('AZYYV',1))
print(rot('AZYYV',13))
print(rot('AZYYV',25))

IFMMP
NMLLI
ZYXXU


## Working code

In [83]:
def pattern(word: str)-> tuple:
    """
    Generates a pattern tuple representing the differences between each adjacent pair of letters in the input word,
    based on their positions in the alphabet.
    Parameters:
    - word (str): The input word from which to generate the pattern.

    Returns:
    - tuple: A tuple of integers representing the differences between each pair of adjacent letters in the input
      word.

    Note: The function is designed to work with any sequence of characters that fall within the ASCII range of valid
    English letters. The output tuple's length is always one less than the length of the input word
    """

    pat = []
    for i in range(len(word) - 1):

        # Calculating the circular difference between adjacent characters
        diff = (ord(word[i])-ord(word[i+1]))%26

        # Adjusting the difference for the minimal circular distance
        pat.append(diff if diff <= 13 else diff - 26)
    return tuple(pat)

In [93]:
def find_rot_equivalence_classes(strings: list[str]) -> list[list[str]]:
  """
  This function groups a list of strings into equivalence classes based on their rotational pattern. Two strings belong
  to the same equivalence class if the differences between each pair of adjacent letters are the same across the strings.

  Parameters:
  - strings (list[str]): A list of strings to be classified into equivalence classes based on their rotational patterns.

  Returns:
  - list[list[str]]: A list where each sublist contains strings that are equivalent in terms of their rotational
    patterns. Strings within the same sublist can be considered rotations of one another, following the same
    pattern of letter shifts.

  Note: The function assumes that all inputs are composed of capital letters from the English alphabet.
  """
  eqv = {}
  for word in strings:
    pat = pattern(word)
    if pat in eqv.keys():
      eqv[pat].append(word)
    else:
      eqv[pat] = [word]

  return list(eqv.values())

In [94]:
#simple test
res = find_rot_equivalence_classes(['HELLO','URYYB','IFMMP','WORLD','ASVPH','SUN'])
print(res)

[['HELLO', 'URYYB', 'IFMMP'], ['WORLD', 'ASVPH'], ['SUN']]


## Unit tests

In [86]:
import unittest
class TestCipherFunctions(unittest.TestCase):

    def test_word_pattern(self):
        # Test the pattern function with a few examples
        self.assertEqual(pattern('HELLO'), (3, -7, 0, -3))
        self.assertEqual(pattern('IFMMP'), (3, -7, 0, -3))
        self.assertEqual(pattern('URYYB'), (3, -7, 0, -3))
        self.assertEqual(pattern('WORLD'), (8, -3, 6, 8))
        self.assertEqual(pattern('ASVPH'), (8, -3, 6, 8))

    def test_find_rot_equivalence_classes_example(self):
        # Test with the example input
        self.assertEqual(find_rot_equivalence_classes(['HELLO', 'IFMMP', 'URYYB']),
                         [['HELLO', 'IFMMP', 'URYYB']])

    def test_find_rot_equivalence_classes_consec(self):
        # Test words with consecutive letters
        self.assertEqual(find_rot_equivalence_classes(['ABCD', 'EFGH', 'IJKL', 'MNOP']),
                         [['ABCD', 'EFGH', 'IJKL', 'MNOP']])

    def test_find_rot_equivalence_classes_multiple(self):
        # Test with multiple equivalence classes
        self.assertEqual(find_rot_equivalence_classes(['HELLO', 'IFMMP', 'URYYB', 'WORLD', 'ASVPH', 'SUN']),
                         [['HELLO', 'IFMMP', 'URYYB'], ['WORLD', 'ASVPH'], ['SUN']])

    def test_find_rot_equivalence_classes_empty(self):
        # Test with an empty list
        self.assertEqual(find_rot_equivalence_classes([]), [])

    def test_find_rot_equivalence_classes_no_equivalence(self):
        # Test with no equivalent words
        self.assertEqual(find_rot_equivalence_classes(['PYTHON', 'JAVA', 'CPLUSPLUS']),
                         [['PYTHON'], ['JAVA'], ['CPLUSPLUS']])

    def test_find_rot_equivalence_classes_same_word(self):
        # Test with the same word repeated
        self.assertEqual(find_rot_equivalence_classes(['HELLO', 'HELLO', 'HELLO']),
                         [['HELLO', 'HELLO', 'HELLO']])

    def test_find_rot_equivalence_classes_end(self):
        # Alphabet interval end letters with small and large difference
        #BAZZW 1, NMLLI 13, ZYXXU 25
        self.assertEqual(find_rot_equivalence_classes(['AAZB', 'ZZYA', 'AZYYV','NMLLI','BAZZW', 'ZYXXU']),
                         [['AAZB', 'ZZYA'], ['AZYYV','NMLLI','BAZZW','ZYXXU']])

    def test_find_rot_equivalence_classes_almost_equivalent(self):
        # Test with words that are close to being Rot-equivalent.
        #'QRSUVW', 'YZACDE' would have been in the same list
        self.assertEqual(find_rot_equivalence_classes(['ABCEFG', 'YZBCDE', 'QRSUVU']),
                         [['ABCEFG'], ['YZBCDE'], ['QRSUVU']])

    def test_find_rot_equivalence_classes_large_one_letter(self):
        # Test with one letter words
        self.assertEqual(find_rot_equivalence_classes(['A', 'B', 'C', 'D', 'E', 'F', 'X', 'Y']),
                         [['A', 'B', 'C', 'D', 'E', 'F', 'X', 'Y']])

    def test_find_rot_equivalence_classes_repeat(self):
        # Test with very large strings to check for performance issues
        long_a = 'A' * 1000
        long_f = 'F' * 1000
        long_axy = 'AXY' * 10000
        long_cza = 'CZA' * 10000
        long_ab = 'AB' * 10000
        self.assertEqual(find_rot_equivalence_classes([long_a, long_f, long_axy, long_cza, long_ab]), [[long_a, long_f], [long_axy, long_cza], [long_ab]])


In [87]:
def run_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestCipherFunctions)
    unittest.TextTestRunner().run(suite)

run_tests()

...........
----------------------------------------------------------------------
Ran 11 tests in 0.064s

OK


## Using input from STDIN

In [104]:
def build_word_list(input_string):
    # Step 1: Remove the leading '[' and trailing ']' from the string
    stripped_input = input_string.strip("[]").strip()

    # Check if the input is empty after stripping
    if not stripped_input:
        return []

    # Step 2: Split the string by commas to get individual words
    words = stripped_input.split(",")

    # Step 3: Remove leading/trailing spaces and apostrophes, then store the cleaned words
    cleaned_words = [word.strip().strip("'") for word in words]

    return cleaned_words

In [105]:
# Read input from STDIN
user_input = input("Please enter a valid input for the Caesar cipher problem:")

# Example usage
function_input = build_word_list(user_input)
user_output = find_rot_equivalence_classes(function_input)
print(user_output)

Please enter a valid input for the Caesar cipher problem:['HELLO', 'IFMMP', 'URYYB', 'WORLD', 'ASVPH', 'SUN']
[['HELLO', 'IFMMP', 'URYYB'], ['WORLD', 'ASVPH'], ['SUN']]
